<a href="https://colab.research.google.com/github/adarsh070/Bluebook_for_bulldozers/blob/main/Bluebook_for_bulldozers_ipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [6]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [7]:
!pip install kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle competitions download -c bluebook-for-bulldozers
!unzip bluebook-for-bulldozers.zip



bluebook-for-bulldozers.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  bluebook-for-bulldozers.zip
replace Data Dictionary.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Data Dictionary.xlsx    
  inflating: Machine_Appendix.csv    
  inflating: Test.csv                
  inflating: Train.7z                
  inflating: Train.zip               
  inflating: TrainAndValid.7z        
  inflating: TrainAndValid.csv       
  inflating: TrainAndValid.zip       
  inflating: Valid.7z                
  inflating: Valid.csv               
  inflating: Valid.zip               
  inflating: ValidSolution.csv       
  inflating: median_benchmark.csv    
  inflating: random_forest_benchmark_test.csv  


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
!pip install fastai

In [11]:

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

from fastai.tabular.all import *


from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn import metrics


from IPython.display import display



In [12]:
!unzip Train.zip

Archive:  Train.zip
  inflating: Train.csv               


In [13]:
PATH = Path('./Train.csv')
df_raw = pd.read_csv(f'{PATH}', low_memory=False, parse_dates=['saledate'])

In [14]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [15]:
add_datepart(df_raw, 'saledate')


In [ ]:

# --- 2. Identify Columns and Define Custom Order ---
custom_order = ['Low', 'Medium', 'High']
cols_with_custom_order = []

for col in df_raw.select_dtypes(include=['object', 'category']).columns:
    unique_vals = set(df_raw[col].dropna().unique())
    if unique_vals == set(custom_order):
        cols_with_custom_order.append(col)

# --- 3. Pre-process Columns with Custom Order ---
for col in cols_with_custom_order:
    df_raw[col] = pd.Categorical(df_raw[col], categories=custom_order, ordered=True)

# --- 4. Define fastai Preprocessing Steps ---
cat_names = list(df_raw.select_dtypes(include=['object', 'category']).columns)
cont_names = list(df_raw.select_dtypes(include=['number']).columns)

if 'ID' in cont_names: cont_names.remove('ID')

procs = [Categorify,FillMissing ]

# --- 5. Apply fastai Preprocessing using TabularPandas ---
y_name = 'SalePrice' # Define your target variable
if y_name in cont_names: cont_names.remove(y_name)
elif y_name in cat_names: cat_names.remove(y_name) # Handle if target is categorical

# Using valid_pct=0.0 ensures all of df_raw is treated as training data for processing
# This creates splits = (all_indices, empty_list)
splits = RandomSplitter(valid_pct=0.0)(range_of(df_raw))

to = TabularPandas(df_raw,
                   procs=procs,
                   cat_names=cat_names,
                   cont_names=cont_names,
                   y_names=y_name,
                   y_block=RegressionBlock() if df_raw[y_name].dtype.kind in 'if' else CategoryBlock(), # Adapt y_block
                   splits=splits)

# --- 6. Access the Processed Data (The entire training set processed) ---
df_raw = to.items



In [28]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/df_raw.feather')

In [24]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()
n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df_raw)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df_raw, n_trn)
y_train, y_valid = split_vals(df_raw.SalePrice, n_trn)
X_train.shape, y_train.shape, X_valid.shape


((389125, 67), (389125,), (12000, 67))

In [25]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def print_score(m):
    res = [rmse(m.predict(X_train), y_train),
           rmse(m.predict(X_valid), y_valid),
           m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

CPU times: user 7min 21s, sys: 170 ms, total: 7min 21s
Wall time: 1min 52s
[2.7564379631393913e-05, 9.074178708482999e-05, 0.9999999984201002, 0.9999999831069084]
